In this notebook, we'll go through some examples of quantum physics simulation packages in Julia. We'll look at tensor network simulation packages, eigensolving packages, and leave with some ideas for projects.

In [ ]:
using LinearAlgebra

## Tensor network simulation

Tensor network methods are a family of techniques commonly employed in many body physics simulation. We saw an example already at this school in DMRG, but there are of course many others (and many types of DMRG), such as PEPS methods, tree tensor networks, MERA, etc.

ITensor is a package that was originally written in C++ but was recently ported from the ground up to Julia. We'll go through some of its features that make it a good tool for physics simulation. Note that there are other tensor network simulation packages in Julia! Jutho Hagaeman has a nice selection on [his GitHub](https://github.com/jutho/).

### Basics: building some tensors, doing operations to them

In [ ]:
# first, let's install the package!
using Pkg; Pkg.add("ITensors")
using ITensors

In [ ]:
# ITensors have indices, which have descriptive "tags"
a = Index(2, "a")
b = Index(3, "b")

In [ ]:
# we can build up an ITensor from the indices
A = ITensor(a, b)
# and set its elements
A[a=>1, b=>3]=2.0

In [ ]:
# the default element type of an ITensor is Float64
@show eltype(A)
# but we can control this
B = ITensor(ComplexF64, b, a)
@show eltype(B)
B[a=>1, b=>3] = 2*im

In [ ]:
# We can contract two tensors using the * operator
A * B

In [ ]:
# ITensors.jl will automatically figure out the output indices for us:
c = Index(2, "c")
d = Index(3, "d")
C = randomITensor(a, b, c)
D = randomITensor(c, d, a)
@show inds(C*D)

In [ ]:
# ITensors also provides decomposition methods like svd and eigen
svd(C, a) # a here is treated as a "left"/"row" index

In [ ]:
svd(C, a, b)

## Building MPS and MPO

In [ ]:
# ITensor provides methods to automatically generate states with
# the correct index flux and dimensionality for various problems
N     = 10
sites = siteinds("S=1/2",N)

In [ ]:
# and we can build MPS from these
ψ     = randomMPS(sites,10)

In [ ]:
# or MPO
O     = randomMPO(sites,5)

In [ ]:
# ITensors will automatically handle matching up the correct "Site" indices
O * ψ

In [ ]:
# We can easily compute inner products
ϕ     = randomMPS(sites,10)
dot(ϕ, O, ψ)

There are quite a few other MPS/MPO methods you can check out:
- `inner`
- `projMPS`
- `projMPO`
- Try constructing with different `siteinds` (e.g. `S=1`, fermions)

## DMRG
We don't want to have to construct the MPO element by element -- how annoying! Instead, `ITensors` provides a way to automatically construct MPOs from equations -- `AutoMPO`:

In [ ]:
N = 10
ampo = AutoMPO()
h = 0.2
for j=1:N-1
    add!(ampo, -1,"Sz",j,"Sz",j+1)
end
for j=1:N
    add!(ampo, h,"Sx",j)
end
# Convert these terms to an MPO tensor network
sites = siteinds("S=1/2",N)
H = MPO(ampo,sites)

In [ ]:
# With this MPO, we can do DMRG
ψ₀ = randomMPS(sites,10)
sweeps = Sweeps(15)
setmaxdim!(sweeps, 10,20,100,100,200) # set the maximum bond dimension at each sweep
setcutoff!(sweeps, 1E-10) # set truncation error cutoff (for all sweeps)
energy, ψ = dmrg(H,ψ₀,sweeps)

In [ ]:
# We can also build "observers" into our DMRG to track convergence over time or observables of interest
Sz_observer = DMRGObserver(["Sz"],sites,energy_tol=1E-7)
energy, ψ = dmrg(H,ψ₀,sweeps, observer = Sz_observer)

Let's take some time to explore. Some suggestions:
- Try the [examples](https://github.com/ITensor/ITensors.jl/tree/master/examples/dmrg) in the ITensors repo (you might find the Hubbard models particularly interesting, or models with fermions).
- Play around with some of the [options](https://itensor.github.io/ITensors.jl/dev/DMRG.html) and varities of DMRG that ITensors supports
- Try changing up the `AutoMPO` to include disorder. You might find [Distributions.jl](https://github.com/JuliaStats/Distributions.jl) helpful for generating certain distributions of random numbers.
- Try the [multithreading](https://itensor.github.io/ITensors.jl/dev/Multithreading.html) support. Do you notice a speed difference?

## Krylov methods

Krylov methods are used to perform eigensolving or time evolution. You likely met them already in the lecture on exact diagonalization. We'll use [KrylovKit.jl](https://github.com/Jutho/KrylovKit.jl) to perform some basic eigensolving. Note again that there are [*quite a few*](https://jutho.github.io/KrylovKit.jl/latest/#Package-features-and-alternatives) Julia packages aiming to solve these types of problems, and you're encouraged to check them out.

In [ ]:
# first, let's install the package!
using Pkg; Pkg.add("KrylovKit")
using KrylovKit

In [ ]:
# KrylovKit provides methods for (extremal) eigenvalue solving, operator exponentiation, SVD, and several others.
# Let's try a simple example with an explicit matrix
n = 1024 # small
A = rand(ComplexF64,(n,n)) .- one(ComplexF64)/2
A = (A+A')/2

In [ ]:
ishermitian(A)

In [ ]:
tol = 10*n*eps(real(ComplexF64))
λ, ϕ, info = eigsolve(A, 1, :SR, orth=ClassicalGramSchmidt(), tol=tol)

In [ ]:
# KrylovKit also supports "matrix-free methods"
# for these we need to provide a starting guess x₀
# Here, I'm going to be lazy and provide a "fake" method
f(v::AbstractVector) = A*v
x₀ = randn(n)
tol = 10*n*eps(real(ComplexF64))
λ, ϕ, info = eigsolve(f, x₀, 1, :SR, orth=ClassicalGramSchmidt(), tol=tol)

In [ ]:
# Now let's try the exponentiator
x = randn(n)
alg = Arnoldi(orth = ClassicalGramSchmidt(), krylovdim = n, maxiter = 2, tol = 10*n*eps(real(ComplexF64)))
y = exponentiate(A, 1, x, alg)

In [ ]:
y = exponentiate(A, 2im/π, x, alg)

In [ ]:
y = exponentiate(f, 2im/π, x, alg)

Let's take some time to explore. Some suggestions:
- Try different solver methods -- SVD, for example
- Try building your own matrices or matrix-free functions using what you learned in David Luitz's lecture
- How do different orthogonalizers affect things?